In [ ]:
from Libs.Profile.stocks import Stock, project_stock_price, plot_stock_value
from Libs.Profile.portfolio import FinancialPortfolio
import os
from Libs.global_vars import STOCKS_HISTORY_PATH

portfolio = FinancialPortfolio('verybigpack')

# Load every file
for file_stock in [f for f in os.listdir(STOCKS_HISTORY_PATH) if f.endswith('.csv')]:
    portfolio.add_stock(Stock(stock_to_load=file_stock))
# print(portfolio)
portfolio.get_ammounts_with_risks()

test = portfolio.get_stock('S&P500')
test.update_stock(stock_prior_risk=0.02)
# test.stock_history['date'] = ['2025-03-06 11:36:22', '2025-04-06 11:36:22', '2025-06-08 00:59:51', '2025-06-06 00:59:51', '2025-06-28 00:59:51', '2025-07-03 00:59:51', '2025-07-08 00:59:51', '2025-07-09 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51', '2025-06-08 00:59:51']
import numpy as np
for i in np.random.normal(50,2,5):
    test.update_stock(stock_price=i)
test.stock_history['date']=['2025-02-06 11:36:22', '2025-03-06 11:36:22', '2025-04-08 01:02:26', '2025-04-28 01:02:26', '2025-05-02 01:02:26', '2025-05-08 01:02:26', '2025-06-04 01:02:26', '2025-06-08 01:02:26']

plot_stock_value(test)


## **Update Movements**

In [ ]:
import polars as pl
import numpy as np
from Libs.Management.bank_api import get_history_movements, update_movements_file
update_movements_file(new_orig_mov_file='data/raw_data/operazioni.xlsx', header_start=17)
get_history_movements().head()

## **Update Balance**

In [ ]:
import polars as pl
import numpy as np
from Libs.Management.bank_api import get_current_balance, get_history_balance, reconstruct_balance, update_balance_from_movements
# reconstruct_balance(current_balance=to_give) # Only for creating the balance history
update_balance_from_movements()
get_history_balance().head()

# **Prediciton from Balance Example**

In [ ]:
from Libs.Models.data_visualization import plot_history_balance, overlay_generic_graph
from Libs.Management.bank_api import get_history_balance
from Libs.Models.data_manipulation import get_regularized_dataset, get_moving_average

# Get Balance, regularize it and take the moving average
balance_df = get_history_balance()
df = get_regularized_dataset(balance_df, reg_period='1d')
df_ma = get_moving_average(df, window_size=15)

# Perform date to int conversion so that we can perform data imputation on the missing vals from moving average
from Libs.Models.data_manipulation import simple_linear_imputation, convert_date_to_numeric, convert_numeric_to_date
df_ma_imputed = convert_date_to_numeric(df_ma, ['date'])
df_ma_imputed = simple_linear_imputation(df_ma_imputed, 'date', 'moving_average')

# Perform a linear fit on the imputed dataset and go back to date format
from Libs.Models.Predictions.algorithms import linear_fit
from Libs.Models.Predictions.algorithms import get_model_prediction

df_ma_imputed, linear_model = linear_fit(dataset=df_ma_imputed, cols_to_fit=None)
df_ma_imputed = convert_numeric_to_date(df_ma_imputed, ['date'])

# Define some new dates for forecasting and perform the prediction with the model created
X = convert_date_to_numeric(['2025-06-01', '2025-07-01', '2025-08-01'])
pred_df = get_model_prediction(X, linear_model)
pred_df = convert_numeric_to_date(pred_df, date_cols=['date'])

# Show the results
plot_history_balance(balance_df=df, additional_plots=[{'plot_function' : overlay_generic_graph,
                                                       'plot_parameters':{'dataset':df_ma, 'col_to_plot':'moving_average',
                                                                          'color':'darkred', 'linestyle':'--','label':'Moving Average'}}])

plot_history_balance(balance_df=df_ma, balance_col='total_value', additional_plots=[
    {'plot_function':overlay_generic_graph, 'plot_parameters':{'dataset':df_ma_imputed, 'col_to_plot':'moving_average', 'label':'moving average (linear imputation)'}},
    {'plot_function':overlay_generic_graph, 'plot_parameters':{'dataset':pred_df, 'col_to_plot':'model_prediction', 'label':'linear prediction', 'marker':'o', 'linestyle':'--'}}
    ]
)



In [ ]:
from Libs.Models.Predictions.algorithms import goal_reach_prediction
goal_reach_prediction(linear_model, '2025-07-01', 15_000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

a = ['2025-11-01', '2025-11-02', '2025-11-3','2025-11-04','2025-11-05']
b = np.random.rand(len(a))*10+np.linspace(0,10, len(a))


def plot_generic_timeseries(dates : list, vals : list, figsize=(12,6), show=True, title=None, label=''):
    """
    """

    numerical_dates = convert_date_to_numeric(dates)[:,0]
    plt.plot(numerical_dates, vals, marker='o', linestyle='--', linewidth=1, color='black', label=label, alpha=0.7)

    if title is not None:
        plt.title(title)

    plt.xlabel('Date')
    plt.ylabel('€')
    plt.xticks(numerical_dates, dates)
    plt.xticks(rotation=45)
    plt.grid(alpha=0.4)
    plt.tight_layout()
    plt.legend()

    if show:
        plt.figure(figsize=figsize)
        plt.show()

plot_generic_timeseries(a, b, show=False)
a = ['2025-11-01', '2025-11-02', '2025-11-3','2025-11-04']
b = np.random.rand(len(a))+np.linspace(0,10, len(a))
plot_generic_timeseries(a, b, show=True, label='tt')
# plt.show()